In [1]:
import pandas as pd
from geopy.distance import lonlat, distance
import numpy as np

In [2]:
git_hub = "C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\"
#apt_geocode = pd.read_csv("C:\\Users\\Young Jin Park\\Documents\\GitHub\\RGTM\\Cleaned//merged_manhattan_geocode.csv")

In [13]:
apt_geocode["GeoCode"] = list(zip(apt_geocode.longitude, apt_geocode.latitude))

In [43]:
def calculate_x_mile_vals(df, thr, value_col, fn, dist_df=None, merge_df=None):
    values=[]
    
    for i in range(len(df)):
        center = df.loc[i,"GeoCode"]
        if dist_df is None:
            temp_df = df.copy(deep=True)
        else:
            temp_df = dist_df.copy(deep=True)
        print(i)
        temp_df = temp_df[temp_df["GeoCode"]!=center].copy(deep=True)
        temp_df["Distance"] = temp_df["GeoCode"].apply(lambda x : distance(lonlat(*center), lonlat(*x)).miles)
        
        temp_df = merge_df.merge(temp_df, how="inner", on ="GeoCode")
        temp_df[value_col] = temp_df[value_col].apply(lambda x : float(x.replace(",","")))
        
        if fn == "avg":
            print(len(temp_df[temp_df["Distance"]<thr]))
            values.append(temp_df[temp_df["Distance"]<thr][value_col].mean())
        elif fn == "count":
            values.append(temp_df[temp_df["Distance"]<thr][value_col].count())
        elif fn == "min_dist":
            values.append(temp_df["Distance"].min())
        else:
            print("fn_not defined. Please pass in avg,count or min_dist")
    
    df["Values"] = values
    
    return df
        
        
    
    

In [50]:
test_df = apt_geocode.loc[:30, :]

In [51]:
test_rslt = calculate_x_mile_vals(test_df, thr=5, value_col="longitude", fn="min_dist")

C:\Users\Young Jin Park\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
apt_df = pd.read_csv(git_hub + "Cleaned\\merged_with_geocode.csv")

In [15]:
distinct_apt = pd.read_csv(git_hub + "Cleaned\\merged_manhattan_geocodes.csv")

In [16]:
feature_df = pd.read_csv(git_hub + "Cleaned\\nycopen_zillow_matched_data.csv")

In [17]:
distinct_apt["StreetAddress"] = distinct_apt["Address"].apply(lambda x : x.split(",")[0])
distinct_apt["GeoCode"] = distinct_apt["GeoCode"].apply(lambda x : eval(x))
apt_df["GeoCode"] = apt_df["GeoCode"].apply(lambda x : eval(x))

In [18]:
feature_df["StreetAddress"] = feature_df.StreetNum + " " + feature_df.StreetName

In [19]:
merged = feature_df.merge(distinct_apt[["StreetAddress", "GeoCode"]], how="left", on="StreetAddress")

In [44]:
merged1 = calculate_x_mile_vals(merged, thr=0.5, value_col="SalePrice", fn="avg", dist_df=distinct_apt, merge_df=apt_df)

0
1852
1
1852
2
1852
3
1005
4
1005
5
1029
6
1329
7
1329
8
1329
9
1275
10
1275
11
1905
12
1905
13
8215
14
8215
15
7490
16
7490
17
7490
18
7490
19
7662
20
7662
21


KeyboardInterrupt: 

In [45]:
merged.to_csv(git_hub+"Cleaned\\FINAL_MERGED_MANHATTAN.csv")